<a href="https://colab.research.google.com/github/NataIss/codeforces/blob/main/Codeforces_15_NumOfUsersByContest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import requests
import pandas as pd
import numpy as np
import time

In [2]:
try:  
    data = pd.read_json("https://codeforces.com/api/contest.list?gym=false")
    con = pd.DataFrame.from_records(data['result'])
    con
except (BaseException) as e:    
    print(e)
con

,id,name,type,phase,frozen,durationSeconds,startTimeSeconds,relativeTimeSeconds
0,1630,Codeforces Round #768 (Div. 1),CF,BEFORE,False,7200,1643294100,-1112451
1,1631,Codeforces Round #768 (Div. 2),CF,BEFORE,False,7200,1643294100,-1112451
2,1628,Codeforces Round #767 (Div. 1),CF,BEFORE,False,7200,1642862100,-680451
3,1629,Codeforces Round #767 (Div. 2),CF,BEFORE,False,7200,1642862100,-680451
4,1626,Educational Codeforces Round 121 (Rated for Di...,ICPC,BEFORE,False,7200,1642343700,-162051
...,...,...,...,...,...,...,...,...
1547,5,Codeforces Beta Round #5,ICPC,FINISHED,False,7200,1269100800,373080849
1548,4,Codeforces Beta Round #4 (Div. 2 Only),ICPC,FINISHED,False,7200,1268395200,373786449
1549,3,Codeforces Beta Round #3,ICPC,FINISHED,False,7200,1267963200,374218449
1550,2,Codeforces Beta Round #2,ICPC,FINISHED,False,7200,1267117200,375064449


In [13]:
con=con[(con['type']=='CF') & (con['phase']=='FINISHED')]
con

,id,name,type,phase,frozen,durationSeconds,startTimeSeconds,relativeTimeSeconds
6,1625,Codeforces Round #765 (Div. 2),CF,FINISHED,False,7200,1641989100,192549
8,1621,Hello 2022,CF,FINISHED,False,8100,1641220500,961148
9,1616,Good Bye 2021: 2022 is NEAR,CF,FINISHED,False,7200,1640792100,1389549
10,1623,Codeforces Round #763 (Div. 2),CF,FINISHED,False,7200,1640698500,1483149
12,1615,Codeforces Global Round 18,CF,FINISHED,False,9000,1640356500,1825149
...,...,...,...,...,...,...,...,...
1524,28,Codeforces Beta Round #28 (Codeforces format),CF,FINISHED,False,7200,1284735600,357446049
1525,27,"Codeforces Beta Round #27 (Codeforces format, ...",CF,FINISHED,False,7200,1284130800,358050849
1526,26,Codeforces Beta Round #26 (Codeforces format),CF,FINISHED,False,7200,1281970800,360210849
1531,21,Codeforces Alpha Round #21 (Codeforces format),CF,FINISHED,False,7200,1277730300,364451349


In [43]:
 link='https://codeforces.com/api/contest.ratingChanges?contestId=1625'
 data = pd.read_json(link)
 if data['status'][0]=='OK':
       dfAll = pd.DataFrame.from_records(data['result'])

In [22]:
for j in con['id']: 
    i=0
    while i<3:
        try:
            time.sleep(0.2)
            link='https://codeforces.com/api/contest.ratingChanges?contestId=' + str(j)
            data = pd.read_json(link)
            if data['status'][0]=='OK':
                df = pd.DataFrame.from_records(data['result'])
                dfAll=pd.concat([dfAll,df],ignore_index=True)
                i=i+5
                print(j) + "****"
        except (BaseException) as e:    
                time.sleep(1)
                i=i+1
                #ratedListAll[ratedListAll['handle']==j][0]
                if i==3:
                    print(j)
                    print(e)
dfAll

1625
1621
1616
1623
1615
1617
1585
1591
1608
1597
HTTP Error 400: 
1609
1614
1610
1584
1588
1589
1605
1596
HTTP Error 400: 
1603
1604
1601
1602
1582
1583
1586
1595
HTTP Error 400: 
1594
1592
1580
1581
1572
0
1573
0
1566
1567
1556
1562
1558
1561
1563
1559
1557
1548
1549
1554
1552
1553
1544
1530
1545
1546
1543
1542
1540
1541
1539
1537
1534
1536
1523
1526
1528
1529
1527
1521
1515
1517
1516
1514
1508
1509
1513
1503
1504
1498
1483
1484
1482
1497
HTTP Error 400: 
1500
1501
1495
1496
1493
1491
1492
1486
1485
1479
1480
1481
1477
1478
1474
1467
1470
1471
1466
1411
1464
1465
1458
1459
1410
HTTP Error 400: 
1461
1450
1453
1415
1456
1457
1414
HTTP Error 400: 
1451
1439
1440
1446
1447
1438
0
1442
1443
1441
1444
1445
1413
1434
1435
1436
1412
HTTP Error 400: 
1421
1428
1427
1422
1408
1416
1417
1420
1419
1406
1407
1404
1405
1396
1397
1401
1392
1394
1395
1391
1393
1388
1383
1384
1381
1382
1379
1372
0
1375
1371
1369
1370
1368
1364
1365
1361
1362
1363
1358
1355
1349
1350
1351
HTTP Error 400: 
1344
0
1345

,contestId,contestName,handle,rank,ratingUpdateTimeSeconds,oldRating,newRating
0,1625,Codeforces Round #765 (Div. 2),1a2b3c4,1,1641996300,1878,2203
1,1625,Codeforces Round #765 (Div. 2),E-Hentai,2,1641996300,1427,1939
2,1625,Codeforces Round #765 (Div. 2),Alan_boyfriend,3,1641996300,903,1552
3,1625,Codeforces Round #765 (Div. 2),ZayinLoveDafeng,4,1641996300,696,1404
4,1625,Codeforces Round #765 (Div. 2),lxr_loves_yxh,5,1641996300,1984,2202
...,...,...,...,...,...,...,...
3465771,26,Codeforces Beta Round #26 (Codeforces format),69-64-79,490,1281978000,1487,1374
3465772,26,Codeforces Beta Round #26 (Codeforces format),Panicbroom,490,1281978000,1500,1377
3465773,26,Codeforces Beta Round #26 (Codeforces format),sock258,490,1281978000,1246,1175
3465774,26,Codeforces Beta Round #26 (Codeforces format),riki,490,1281978000,1215,1148


In [23]:
dfAll.drop_duplicates(keep='last',subset=['handle','contestId'],inplace=True)
dfAll

,contestId,contestName,handle,rank,ratingUpdateTimeSeconds,oldRating,newRating
49199,1625,Codeforces Round #765 (Div. 2),1a2b3c4,1,1641996300,1878,2203
49200,1625,Codeforces Round #765 (Div. 2),E-Hentai,2,1641996300,1427,1939
49201,1625,Codeforces Round #765 (Div. 2),Alan_boyfriend,3,1641996300,903,1552
49202,1625,Codeforces Round #765 (Div. 2),ZayinLoveDafeng,4,1641996300,696,1404
49203,1625,Codeforces Round #765 (Div. 2),lxr_loves_yxh,5,1641996300,1984,2202
...,...,...,...,...,...,...,...
3465771,26,Codeforces Beta Round #26 (Codeforces format),69-64-79,490,1281978000,1487,1374
3465772,26,Codeforces Beta Round #26 (Codeforces format),Panicbroom,490,1281978000,1500,1377
3465773,26,Codeforces Beta Round #26 (Codeforces format),sock258,490,1281978000,1246,1175
3465774,26,Codeforces Beta Round #26 (Codeforces format),riki,490,1281978000,1215,1148


In [24]:
w=pd.DataFrame(dfAll.groupby('contestId')['rank'].count())
w.head()

,rank
contestId,
26,509
27,389
28,379
29,355
30,570


In [25]:
w

,rank
contestId,
26,509
27,389
28,379
29,355
30,570
...,...
1616,14127
1617,9733
1621,15425


In [26]:
con.set_index('id',inplace=True)

In [27]:
res = pd.merge(con, w, how="outer",left_index=True,right_index=True)
res

,name,type,phase,frozen,durationSeconds,startTimeSeconds,relativeTimeSeconds,rank
20,Codeforces Alpha Round #20 (Codeforces format),CF,FINISHED,False,5400,1276875000,365306649,NaN
21,Codeforces Alpha Round #21 (Codeforces format),CF,FINISHED,False,7200,1277730300,364451349,NaN
26,Codeforces Beta Round #26 (Codeforces format),CF,FINISHED,False,7200,1281970800,360210849,509.0
27,"Codeforces Beta Round #27 (Codeforces format, ...",CF,FINISHED,False,7200,1284130800,358050849,389.0
28,Codeforces Beta Round #28 (Codeforces format),CF,FINISHED,False,7200,1284735600,357446049,379.0
...,...,...,...,...,...,...,...,...
1616,Good Bye 2021: 2022 is NEAR,CF,FINISHED,False,7200,1640792100,1389549,14127.0
1617,Codeforces Round #761 (Div. 2),CF,FINISHED,False,7200,1639661700,2519948,9733.0
1621,Hello 2022,CF,FINISHED,False,8100,1641220500,961148,15425.0
1623,Codeforces Round #763 (Div. 2),CF,FINISHED,False,7200,1640698500,1483149,10110.0


In [35]:
res=res.fillna(0)
res=res[res['rank']>0]

In [36]:
res

,name,type,phase,frozen,durationSeconds,startTimeSeconds,relativeTimeSeconds,rank
26,Codeforces Beta Round #26 (Codeforces format),CF,FINISHED,False,7200,1281970800,360210849,509.0
27,"Codeforces Beta Round #27 (Codeforces format, ...",CF,FINISHED,False,7200,1284130800,358050849,389.0
28,Codeforces Beta Round #28 (Codeforces format),CF,FINISHED,False,7200,1284735600,357446049,379.0
29,"Codeforces Beta Round #29 (Div. 2, Codeforces ...",CF,FINISHED,False,7200,1284994800,357186849,355.0
30,Codeforces Beta Round #30 (Codeforces format),CF,FINISHED,False,7200,1285340400,356841249,570.0
...,...,...,...,...,...,...,...,...
1616,Good Bye 2021: 2022 is NEAR,CF,FINISHED,False,7200,1640792100,1389549,14127.0
1617,Codeforces Round #761 (Div. 2),CF,FINISHED,False,7200,1639661700,2519948,9733.0
1621,Hello 2022,CF,FINISHED,False,8100,1641220500,961148,15425.0
1623,Codeforces Round #763 (Div. 2),CF,FINISHED,False,7200,1640698500,1483149,10110.0


In [37]:
res.rename(columns={'rank':'NumOfUsers'}, inplace=True)

In [38]:
res1=res[['name','startTimeSeconds','NumOfUsers']]
res1.head()

,name,startTimeSeconds,NumOfUsers
26,Codeforces Beta Round #26 (Codeforces format),1281970800,509.0
27,"Codeforces Beta Round #27 (Codeforces format, ...",1284130800,389.0
28,Codeforces Beta Round #28 (Codeforces format),1284735600,379.0
29,"Codeforces Beta Round #29 (Div. 2, Codeforces ...",1284994800,355.0
30,Codeforces Beta Round #30 (Codeforces format),1285340400,570.0


In [41]:
res1.to_excel('NumOfUsersByContest.xlsx')